# 4.1 문자 문제

파이썬3의 `str`에서 가져오는 항목은 유니코드 문자다. 유니코드 표준은 문자의 단위 원소와 특정 바이트 표현을 구분한다.

* 문자의 단위 원소(code point) : 10진수 0에서 1,114,111까지의 숫자이며, 유니코드 표준에서는 `U+` 접두사를 붙여 4자리에서 6자리 사이의 16진수로 표현

* 문자를 표현하는 실제 바이트는 사용하는 인코딩(코드 포인트를 바이트 시퀀스로 변환하는 알고리즘, 반대는 디코딩)에 따라 달라짐

In [23]:
b = b'caf\xc3\xa9'
b # bytes 리터럴은 접두사 b로 시작한다.

b'caf\xc3\xa9'

In [21]:
s = b.decode('utf8')
s # 네 개의 유니코드 문자열

'café'

In [19]:
#s = 'cafe'
len(s)

4

In [24]:
b = s.encode('utf8')
b, len(b)# e`가 UTF-8 에서 두 바이트로 인코딩됨

(b'caf\xc3\xa9', 5)

# 4.2 바이트에 대한 기본 지식

이진 시퀀스를 위해 사용되는 내장 자료형은 `bytes`와 `bytearray` 두 가지가 있다.
* `bytes` : 파이썬 3에서 소개된 불변형
* `bytearray` : 파이썬 2.6에 추가된 가변형

`bytes`와 `bytearray`에 들어 있는 각 항목은 0에서 255 사이의 정수이다. 그러나 이진 시퀀스를 슬라이싱하면 언제나 동일한 자료형의 이진 시퀀스가 만들어진다. (이게 무슨 말이지...)

In [25]:
cafe = bytes('café', encoding='utf_8')
cafe

b'caf\xc3\xa9'

In [31]:
cafe[0] # 각 항목은 range(256)에 들어가는 정수

99

In [33]:
cafe[:1] # bytes는 슬라이싱해도 bytes이다. 슬라이스가 한 바이트일 때도 마찬가지

b'c'

In [28]:
cafe_arr = bytearray(cafe)
cafe_arr

bytearray(b'caf\xc3\xa9')

In [29]:
cafe_arr[-1:]

bytearray(b'\xa9')

`s[0] == s[:1]`이 되는 시퀀스형은 `str`이 유일하다. 그외 모든 시퀀스의 경우, `s[i]`는 항목 하나를, `s[i:i+1]`은 안에 `s[i]` 항목을 가진 동일한 자료형의 시퀀스를 반환한다.

In [42]:
test = str('abcdefg')
test[0], test[:1]

('a', 'a')

In [36]:
test[:1] == test[0]

True

In [39]:
test = list('abcdefg')
test[0], test[:1]

('a', ['a'])

In [41]:
test[0] == test[:1] 

False

이진 시퀀스가 실제로 정수형의 시퀀스이긴 하지만, 실제로는 아스키 텍스트가 들어가는 경우가 많다. 따라서 각 바이트 값에 따라 다음과 같이 세 가지 형태로 출력된다.
* 화면에 출력 가능한 아스키 문자(공백에서 ~까지)는 아스키 문자 그대로 출력
* 탭, 개행 문자, 캐리지 리턴, 백슬레시(\)는 이스케이프 시퀀스(\t, \n, \r, \\)로 출력
* 그외의 값은 널 바이트를 나타내는 \x00처럼 16진수 이스케이프 시퀀스로 출력

`bytes`와 `bytearray`는 포매팅하는 `format()`, `format_map()` 메서드를 제외하고는 `str`이 제공하는 메서드를 모두 지원하며, `casefold()`, `isdecimal()`, `isidentifier()`, `isnumeric ()`, `isprintable()`, `encode()` 등 유니코드 데이터에 관련한 메서드도 지원한다. 또한 이진 시퀀스는 `fromhed()`라는 `str`에 없는 클래스 메서드를 제공하여 공백으로 구분된 16진수 쌍을 파싱해서 이진 시퀀스를 만들 수 있다.

In [44]:
test = bytes.fromhex('31 4B CE A9')
test

b'1K\xce\xa9'

In [45]:
test.decode('utf8')

'1KΩ'

`bytes`와 `bytearray` 객체는 생성자에 다음 인수들을 이용해서 생성 가능
* `str`과 `encoding` 키워드 인수
* 0에서 255 사이의 값을 제공하는 반복 가능형
* 하나의 정수 인수, 인수로 받은 정수 개수만큼의 널 바이트로 초기화된 이진 시퀀스 생성
* `bytes`, `bytearray`, `memoryview`, `array.array` 등 버퍼 프로토콜을 구현하는 객체(원본을 복사해서 새로 생성) : 형변환이 필요할 수도 있음

In [46]:
test = bytes(6)
test

b'\x00\x00\x00\x00\x00\x00'

In [47]:
test = bytes([6])
test

b'\x06'

In [50]:
import array
numbers = array.array('h', [-2,-1,0,1,2]) 
# 'h' 타입 코드는 short int(16비트) 형의 배열을 생성한다.
numbers

array('h', [-2, -1, 0, 1, 2])

In [52]:
octets = bytes(numbers) # numbers를 구성하는 바이트들의 사본을 가지고 있다.
octets # 다섯 개의 short int 형을 나타내는 10바이트

b'\xfe\xff\xff\xff\x00\x00\x01\x00\x02\x00'

## 4.2.1 구조체와 메모리 뷰

`struct` 모듈은 패킹된 바이트를 다앙햔 형의 필드로 구성된 튜플로 분석하고, 이와 반대로 튜플을 패킹된 바이트로 변환하는 함수를 제공한다. `struct`는 `bytes`, `bytearray`, `memoryview` 객체와 함께 사용된다. 

다음은 `memoryview`와 `struct`를 사용해서 GIF 이미지의 너비와 높이를 추출한다.

In [66]:
import struct
fmt = '<3s3sHH' # struct 포맷을 지정. <는 리틀엔디언(?), 3s3s는 3바이트 시퀀스, HH는 16비트 정수 두개를 나타냄
with open('huh2.gif', 'rb') as fp:
    img = memoryview(fp.read())

In [67]:
header = img[:10]
header

In [68]:
bytes(header)

b'GIF89aX\x02O\x01'

In [69]:
struct.unpack(fmt, header) # 종류, 버전, 너비, 높이 튜플로 언패킹

(b'GIF', b'89a', 600, 335)

In [70]:
del header
del img

# 4.3 기본 인코더/디코더

텍스트를 바이트로 혹은 바이트를 텍스트로 변환하기 위해 파이썬은 약 100여 개의 코덱(인코더/디코더)이 포함되어 있다.

In [71]:
for codec in ['latin_1', 'utf8', 'utf16']:
    print(codec, 'El Niño'.encode(codec), sep='\t')

latin_1	b'El Ni\xf1o'
utf8	b'El Ni\xc3\xb1o'
utf16	b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'


In [79]:
for codec in ['ascii','latin_1','cp1252','utf8','utf16','utf-16le']:
    print(codec, 'A'.encode(codec), sep='\t')

ascii	b'A'
latin_1	b'A'
cp1252	b'A'
utf8	b'A'
utf16	b'\xff\xfeA\x00'
utf-16le	b'A\x00'


* latin1(iso8859_1) : 다른 인코딩 및 유니코드의 기반이 되는 중요한 인코딩
* cp1252 : MS에서 둥근 따옴표 및 유로화 기호 등을 추가해서 latin1을 확장
* cp437 : 상자를 그리기 위한 문자를 포함해서 원래 IBM PC에서 사용하는 문자셋
* gb2312 : 중국 본토에서 사용하는 간체를 인코딩하기 위한 레거시 표준
* utf-8 : 웹에서 8비트 인코딩을 하기 위해 가장 널리 사용되는 인코딩 방식
* utf-16le : 16비트 인코딩 체계인 UTF-16의 한 형태

# 4.4 인코딩/디코딩 문제 이해하기

`UnicodeError`라는 범용 예외가 있지만, 대부분 `UnicodeEncodeError`(`str`을 이진 시퀀스로 변환할 때)나 `UnicodeDecodeError`(이진 시퀀스를 `str`로 읽어 들일 때) 같은 구체적인 예외가 발생한다. 또한 소스 코드가 예기치 않은 방식으로 인코딩되어 있으면 `SyntaxError`가 발생하기도 한다.

### 4.4.1 UnicodeEncodeError 처리하기

대부분의 비UTF 코덱은 유니코드 문자의 일부만 처리할 수 있다. 텍스트를 바이트로 변환할 때 문자가 대상 인코딩에 정의되어 있지 않으면 `UnicodeEncodeError`가 발생한다.

In [81]:
city = 'São Paulo'
city.encode('utf-8')

b'S\xc3\xa3o Paulo'

In [82]:
city.encode('utf_16')

b'\xff\xfeS\x00\xe3\x00o\x00 \x00P\x00a\x00u\x00l\x00o\x00'

In [83]:
city.encode('iso8859_1')

b'S\xe3o Paulo'

In [84]:
city.encode('cp437')

UnicodeEncodeError: 'charmap' codec can't encode character '\xe3' in position 1: character maps to <undefined>

In [88]:
city.encode('cp437', errors='ignore') # 인코딩할 수 없는 문자를 건너뛴다. 좋지 않다.

b'So Paulo'

In [89]:
city.encode('cp437', errors='replace') # 물음표(?)로 치환한다.

b'S?o Paulo'

In [90]:
city.encode('cp437', errors='xmlcharrefreplace') # XML 객체로 치환한다.

b'S&#227;o Paulo'

## 4.4.2 UnicodeDecodeError 처리하기

이진 시퀀스를 텍스트로 변환할 때 정당한 문자로 변환할 수 없으면 `UnicodeDecodeError`가 발생한다. 그렇지만 'cp1252' 등 많은 레거시 8비트 코덱은 에러를 발생시키지 않고 바이트 스트림으로 디코딩할 수 있다.

In [96]:
octets = b'Montr\xe9al' # latin1 으로 인코딩
octets.decode('cp1252') # cp1252는 latin1의 슈퍼셋이므로 제대로 디코딩됨

'Montréal'

In [97]:
octets.decode('iso8859_7') # 그리스 문자를 위한 코덱이므로 엉뚱하게 해석함

'Montrιal'

In [98]:
octets.decode('koi8_r') # 러시아어를 위한 코덱

'MontrИal'

In [94]:
octets.decode('utf8')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 5: invalid continuation byte

In [99]:
octets.decode('utf8', errors='replace') # �는 알 수 없는 문자를 표현하기 위해 사용하는 유니코드 공식 치환 문자

'Montr�al'

## 4.4.3 예상과 달리 인코딩된 모듈을 로딩할 때 발생하는 SyntaxError

파이썬 3부터는 UTF-8을 소스 코드 기본 인코딩 방식으로 사용하고 있다. 따라서 인코딩 선언 없이 비UTF-8로 인코딩된 .py 모듈을 로딩하면 에러가 발생한다. 이 문제는 파일 꼭대기에 `coding` 주석을 달아서 해결할 수 있다.

```python
# coding: cp1252

print('Olá Mundo!')
```

## 4.4.4 바이트 시퀀스의 인코딩 방식을 알아내는 방법

바이트 시퀀스의 인코딩 방식을 알아내는 방법은 없다. 따라서 별도로 인코딩 정보를 가져와야 한다. 다만 경험과 통계를 통해 추정할 수는 있다. 이를 탐지하는 `Chardet` 패키지를 이용하면 도움이 된다.

> chardetect filename

In [106]:
!chardetect README.md

README.md: utf-8 with confidence 0.99


## 4.4.5 BOM: 유용한 깨진 문자

인코딩된 텍스트의 이진 시퀀스는 인코딩에 대한 정보를 명시적으로 전달하지 않지만, UTF 포맷은 텍스트 앞에 바이트 순서 표시(byte order mark, BOM)을 추가할 수 있다.

In [107]:
u16 = 'El Niño'.encode('utf16')
u16

b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'

b'\xff\xfe' 문자가 바로 바이트 순서 표시로, 인코딩한 인텔 CPU의 '리틀엔디언' 바이트 순서를 나타낸다. ?? 무슨 말이지

# 4.5 텍스트 파일 다루기